In [ ]:
!pip install pymongo

Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу. <br>
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
import json

In [3]:
from pymongo import MongoClient

In [4]:
from pprint import pprint

In [5]:
from time import sleep

In [6]:
def get_vacancies_soup():
    response = requests.get(url, headers=headers, params=params)    
    soup = bs(response.content, 'html.parser')      
    articles = soup.find_all('div', {'class': 'vacancy-preview-card__top'})
    
    return articles

In [84]:
def get_vacancies_list(articles):
    vacancies_list = []
    for article in articles:
        money = article.find('div', {'class': 'vacancy-preview-card__salary'}).get_text(strip=True).replace('\xa0', '')
        if money != 'по договоренности':
            money = money.replace('—', '').split(' ')
            for i in range(len(money)):
                try:
                    money[i] = int(money[i])
                except ValueError:
                    pass
        vacancies_list.append(
            {
            'name': article.find('h3', {'class': 'vacancy-preview-card__title'}).get_text(strip=True),
            'salary': money,
            'link': head_link + article.find('a', {'class': 'vacancy-preview-card__title_border'})['href'],
            'employer': article.find('span', {'class': 'vacancy-preview-card__company-name'}).get_text(strip=True),
            'location': article.find('span', {'class': 'vacancy-preview-location__address-text'}).get_text(strip=True),
            'short review': article.find('div', {'class': 'vacancy-preview-card__short-description'}).get_text(strip=True).replace('\u200b', '')
            })
    return vacancies_list

In [8]:
def upgrade_vacansies_database():
    articles = get_vacancies_soup()
    vacancies_list = get_vacancies_list(articles)
    for vacanсy in vacancies_list:
        if len(list(vacancies_db.vacancies.find({'short review': {'$eq': vacanсy['short review']}}))) == 0:
            vacancies_db.vacancies.insert_one(vacanсy)

In [9]:
head_link = 'https://spb.rabota.ru'

In [10]:
client = MongoClient('127.0.0.1', 27017)

In [11]:
vacancies_db = client['vacancies_db']

In [89]:
#upgrade_vacansies_database()

In [97]:
%%time
for i in range(3):
    url = 'https://spb.rabota.ru/vacancy'
    params = {
        'query': 'python',
        'sort': 'relevance',
        'page': i
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.2.931 Yowser/2.5 Safari/537.36'
    }
    upgrade_vacansies_database()
    sleep(5) #на трех секундах перерыва банит..... да и на пяти тоже банит. Надо меньше страниц за раз листать

Wall time: 22.8 s


In [91]:
for i in vacancies_db.vacancies.find():
    pprint(i)

{'_id': ObjectId('63efa74158c893947481db43'),
 'employer': 'ООО "МАРКЕТ-МОБАЙЛ"',
 'link': 'https://spb.rabota.ru/vacancy/46968606/',
 'location': 'м. Приморская',
 'name': 'Продавец-консультант салона связи Теле2',
 'salary': [35000, 140000, 'руб.'],
 'short review': 'Приглашаем кандидатов на должность продавца-консультанта '
                 'салона связи; Условия: График 2/2, 5/2, 3/3 или на выбор; '
                 'Официальное оформление на работу; Оплата стажировки и '
                 'обучения 700р.) Бесплатная мобильная связь; Рабочее место '
                 'рядом с домом (разные районы города); Выплаты заработной '
                 'платы 2 раза в месяц…'}
{'_id': ObjectId('63efa74158c893947481db44'),
 'employer': 'ИНДИВИДУАЛЬНЫЙ ПРЕДПРИНИМАТЕЛЬ ЛУЦЕНКО НИКИТА АЛЕКСАНДРОВИЧ',
 'link': 'https://spb.rabota.ru/vacancy/46969078/',
 'location': 'м. Ладожская',
 'name': 'Менеджер по работе с клиентами',
 'salary': ['от', 45000, 'руб.'],
 'short review': 'Общаться с потенциальным

In [35]:
def get_salary_more_then(threshold):
    i = list(vacancies_db.vacancies.find({'salary': {"$ne": 'по договоренности', '$gt': threshold}}))
    return i

In [100]:
a = get_salary_more_then(150000)
a

[{'_id': ObjectId('63efa7f658c893947481db6f'),
  'name': 'Менеджер по продажам',
  'salary': [70000, 200000, 'руб.'],
  'link': 'https://spb.rabota.ru/vacancy/46756606/',
  'employer': 'ООО "Электронные системы"',
  'location': 'м. Электросила',
  'short review': 'Работа с клиентами по оборудованию из каталога; Развитие продаж линейки продукции; Консультирование клиентов по ассортименту продукции, подбор аналогов; Наполнение контента сайта по продукции; Работа с программой 1C CRM (выписка счетов, накладных, выставление КП, заключение договоров) Выполнение плана…'},
 {'_id': ObjectId('63efa7f658c893947481db71'),
  'name': 'Бренд-менеджер',
  'salary': [70000, 200000, 'руб.'],
  'link': 'https://spb.rabota.ru/vacancy/46764563/',
  'employer': 'ООО "Электронные системы"',
  'location': 'м. Электросила',
  'short review': 'Развитие продаж линейки продукции; Консультирование клиентов по ассортименту продукции, подбор аналогов; Наполнение контента сайта по продукции; Работа с программой 1C C

In [98]:
vacancies_db.vacancies.count_documents({})

54